In [3]:
!pip indtal  arxiv

ERROR: unknown command "indtal" - maybe you meant "install"



In [18]:

# this is the agent for which the data source is wikipedia for the agent

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [19]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper= api_wrapper)

In [6]:
# data source = a webbsed loader -> website
# A custom tool
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings        #OpenAIEmbeddings  # use Ollamaembedig if not openai api

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings() )
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024FCF9FF990>)

In [7]:
# retriever
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(retriever, "langsmith_search",
                      "search for information about Langsmith.For any question about langsmith,you must use this tool")

In [8]:
retrieval_tool.name

'langsmith_search'

In [20]:
# tool 3
# Arxiv tool inbuilt by langchain
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [21]:
# combining
tools = [wiki,retrieval_tool,arxiv]

import os
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-3.5-turbo=0125",temperature = 0)


In [22]:
from langchain import hub
# get the prompt 
prompt  = hub.pull("hwchse17/openai-functions-agent")
prompt.messages

LangSmithUserError: API key must be provided when using hosted LangSmith API

In [30]:
# querying from these tools -> using agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

NameError: name 'prompt' is not defined

In [31]:
# agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent,tools=tools,verbose = True)

NameError: name 'agent' is not defined

In [32]:
agent_executor.invoke({"input": "Tell me about Langsmith"})

NameError: name 'agent_executor' is not defined